# Tarea 2 Inteligencia Artificial: Procesamiento de lenguaje natural.

### Integrantes: José Castillo, Pablo Silva, y Mario Rozas.

Primero, instalamos lo necesario, en este caso la libreria spacy y modelos de embedding basados en método del tipo Word2Vec.

In [1]:
!pip install spacy
!pip install nltk
!python -m spacy download es_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-10-09 02:59:40.046835: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.9 MB 32.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


Luego importamos las librerias y funciones necesarias para el funcionamiento del programa.

In [2]:
import es_core_news_sm
from string import punctuation
from gensim.models import Word2Vec, KeyedVectors
import numpy as np
import pandas as pd
import regex
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity
import os
from spacy.lang.es.stop_words import STOP_WORDS
from string import punctuation
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from difflib import SequenceMatcher

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Vamos a definir ciertas funciones que ayudarán a hacer lo que se necesita.

**EntrenarModelo(oraciones,NombreModelo)**, permite entrenar un modelo Word2Vec a partir de un conjunto de oraciones extraída. El modelo generado se graba en la carpeta **NombreModelo**. Asumimos que la ventaja de entrenamiento es 2 ($windows=2$) y que el número de dimensiones o tamaño del vector es 700 ($size=700$). (*Esto determinado en base a ensayo y error al final del collab*).

**CargarModelo(NombreModelo)**, que permitirá cargar un modelo cuando sea necesario.

In [66]:
def EntrenarModelo(oraciones,NombreModelo):
    model = Word2Vec(oraciones, size=700, window=2, min_count=1)
    model.save(NombreModelo)
    
def CargarModelo(NombreModelo):
   modelo = Word2Vec.load(NombreModelo)
   vocabulario = [term for term in modelo.wv.vocab]  
   return(modelo,vocabulario)

**ObtenerEmbeddingOracion(modelo, oracion)**, permitirá obtener el embedding (vector) de una oración desde un modelo entrenado. El embedding de una oración es simplemente el vector promedio de cada una de las palabras de la oración: 

In [5]:
def ObtenerEmbeddingOracion(modelo, oracion):
   Lista_vectores = []
   for w in Tokenizar(oracion):
       # Verificar que la palabra w exista en el modelo
       try:
           modelo.wv[w]
       except KeyError:
           continue
       # Obtener vector de la palabra
       vec = modelo.wv[w]
       Lista_vectores.append(vec)
   embedding_palabras = np.array(Lista_vectores)
   if (len(embedding_palabras) > 0):
        embedding_oracion = embedding_palabras.mean(axis=0)
   else:
        embedding_oracion = np.zeros(modelo.vector_size)
   return(embedding_oracion) 

Ahora, utilizamos algunas funciones de preprocesamiento:

**PreProcesarOraciones(textos)**, permite tomar discursos ('textos') y dividirlo en oraciones.

**Tokenizar(oracion)**, toma la oracion y la separa en palabras.

**CrearCorpus(path)**, desde el directorio indicado ('path'), toma los textos que estén en este, y crea una lista con cada texto por separado. Además guarda el nombre de cada archivo.

**CrearDiccionario(lista, claves)**, Se le da el corpus ('lista') y los nombres de los archivos. Permite buscar los archivos por nombre y no por posición.

In [6]:
def PreProcesarOraciones(textos):
    texto_limpio = []
    for texto in textos:  
        if len(texto)!=0:
          texto = regex.sub(' +', ' ', texto)
          tokens = Tokenizar(texto)
          texto_limpio.append(tokens)
    return(texto_limpio)

def Tokenizar(oracion):
    doc = nlp(oracion)
    tokens = [palabra.text for palabra in doc]
    return(tokens)

def CrearCorpus(path):
  directorio = os.listdir(path)
  corpus = []
  doc_id = []  
  for filename  in directorio:
     texto = open(path+filename,'r',encoding="UTF-8").read()
     corpus.append(texto)
     doc_id.append(filename)
  return(corpus,doc_id)

def CrearDiccionario(lista,claves):
  dicc = {}
  for  v in range(0,len(claves)):
    dicc[claves[v]] = lista[v]
  return(dicc)

### Iniciamos el programa

Primero le indicamos la dirección donde se encuentran los textos que se usarán para entrenar el modelo.

Luego cargamos nlp.

Creamos el corpus con los archivos indicados en el PATH.

Con el corpus creado, se preprocesan los textos para dividir las oraciones de estos en listas de palabras.

Luego se crea el diccionario del corpus.

In [7]:
PATH = "/content/DiscursosOriginales/"
nlp          = es_core_news_sm.load()
corpus,docID = CrearCorpus(PATH)
oraciones    =  PreProcesarOraciones(corpus)
CorpusConClave  = CrearDiccionario(corpus,docID)

Entrenamos el modelo. Se le da una lista que contiene cada texto divido en palabras, y el nombre del modelo.

In [67]:
EntrenarModelo(oraciones,'mi_word2vec')

Se carga el modelo.

In [68]:
modelo, vocabulario = CargarModelo('mi_word2vec')

## Función Solicitada (Summarize)

Para implementar la función Summarize, primero se deben hacer ciertas funciones y calcular ciertas cosas.

### Centroide
Primero calculemos el centroide del modelo. 

La función obtiene los embeddings de todos los textos, y los guarda en una lista. Luego se recorre esta lista y se suman todos los vectores (posición a posición), para luego dividirlo en el total de textos, que corresponde al promedio de los vectores, lo cual corresponde al centroide del modelo.

In [69]:
def centroide(modelo):
  vectores = []
  for i in docID:
    doc = CorpusConClave[i]
    vec = ObtenerEmbeddingOracion(modelo, doc)
    vectores.append(vec)

  suma_vec = vectores[0] - vectores[0]
  for j in vectores:
    suma_vec += j
  mean_vec = suma_vec/len(vectores)
  #print(mean_vec)
  return mean_vec

mean_vec = centroide(modelo)

### Puntaje de oraciones

Vamos a determinar para cada oración un puntaje, que corresponde a que tanta similitud tienen sus vectores respecto al centroide antes definido.

Para esto le damos un texto (con su docID). La función separará el texto en oraciones. Luego con un ciclo recorremos estás oraciones, se calcula su embedding con el modelo entrenado, y por ultimo se calcula la distancia respecto al centroide.

La función devuelve el vector con los puntajes de cada oración.

In [47]:
def Obtener_Puntajes(docID):
  vector_distance = []

  token_doc_r = nltk.sent_tokenize(CorpusConClave[docID])
  for j in token_doc_r:
    vector_distance.append(1 - cosine(mean_vec, ObtenerEmbeddingOracion(modelo, j)))

  return vector_distance

### Ordenar Oraciones

Luego de definir los puntajes, ahora las oraciones del texto se ordenarán de mayor a menor según estos puntajes.

Para esto se crea la función Ordenar_Oraciones, que se le da O, que corresponde a la lista de oraciones de un texto, y P, la lista con sus respectivos puntajes.

Luego estos se pasan a un Dataframe, se ordenan de mayor a menor, luego se pasan a una lista las oraciones ordenadas, y se retorna esta lista.

In [15]:
def Ordenar_Oraciones(O, P):
  diccionary = {'Words':O,'vector_distance':P}
  df = pd.DataFrame(diccionary)
  df = df.sort_values(by=['vector_distance'], ascending = False)
  O = df['Words'].tolist()
  O = list(dict.fromkeys(O))
  return O

### Obtener Vector

Función que devuelve el vector de posición (embedding) de una oración. Si bien ya existe está función, por formalidad se crea.

In [16]:
def Obtener_Vector(O):
  vector = ObtenerEmbeddingOracion(modelo, O)
  return vector

### Comparar similitud 2 oraciones

Función que compara 2 embeddings.

In [17]:
def Similitud(Vo,Vr):
  distance = 1 - cosine(Vo, Vr)
  return distance

### Ver si un elemento está en la lista

Función que compara si en una lista de oraciones ('texto'), se encuentra la oración dada ('oracion'). Devuelve True en caso de que ya exista la oración en la lista, False en caso contrario.

In [18]:
def comp_oraciones(texto,oracion):
  for oracion_text in texto:
    ratio = SequenceMatcher(None, oracion_text,oracion).ratio()
    if (ratio==1):
      return True
      break
    else:
      continue
  return False

### Generar Resumen

Con todo lo definido anteriormente, se puede crear la función que generará el resumen de algún texto.

La función es **Generar_Resumen(O, P, N, U)**. Está irá determinando si las oraciones de un texto van o no dentro del resumen pedido. El criterio para incluirlas es que tengan cohesión entre si las oraciones, para eso se usa U, con lo que se irán comparando las oraciones, si una oración cumple con U respecto a la ultima oración incluida en el resumen, se agrega.

O : Corresponde a las oraciones de un texto

P : Corresponde a los puntajes de las oraciones del texto

N : Corresponde al número de oraciones que se busca que contenga el resumen generado

U : Corresponde al nivel de similitud que deben tener las oraciones contenidas en el resumen.

In [32]:
def Generar_Resumen(O, P, N, U):
  O = Ordenar_Oraciones(O,P) #Se ordenan las oraciones de mayor a menor según su puntaje. Lista de oraciones.

  largo = 0 #Atributos de apoyo
  m = len(O)
  Resumen = [] #Lista que irá guardando las oraciones

  for i in range(0, m): #Lista que recorrerá la lista de las oraciones del texto.

    if (largo > N-1): #Si el largo es mayor al largo de la lista, se devuelve el resumen.
      return Resumen

    Vo = Obtener_Vector(O[i]) #Se obtiene el embedding de la oración en la posición i de la lista.

    incluirOracion = False #Variable que determina si se agrega o no la oración al resumen.

    if largo == 0: #En caso de que sea la primera iteración, se agrega la oración, ya que al ser la primera, es la más cercana al centroide y no se puede comparar ya que resumen está vacío.
        incluirOracion = True

    else: #En caso que no sea la primera iteración, se entra aquí.
      for j in range(0, largo): #Ciclo que recorre el largo de la lista resumen.

        Vr = Obtener_Vector(Resumen[j]) #Se obtiene el embedding de la oración en la posición j del resumen.
        Sim = Similitud(Vo,Vr) #Se compara similitud entre Vr y Vo.

        if (Sim > U) and comp_oraciones(Resumen, O[i]) == False: #Si la similitud es mayor a U y la oración no está en la lista resumen, se agrega.
          incluirOracion = True

    if incluirOracion == True: #Si se debe agregar la oración (incluirOracion = True), se añade a la lista resumen y se suma uno a largo.
      Resumen.append(O[i])
      largo += 1

Ahora veamos como funciona con un texto.

In [79]:
O = nltk.sent_tokenize(CorpusConClave['100020.txt'])
P = Obtener_Puntajes('100020.txt')
N = 20
U = 0.9
resumen = Generar_Resumen(O, P, N, U)

Mostramos el resumen.

In [77]:
for i in resumen:
  print(i)

Y, a veces, en Chile cuidamos mucho lo que está dentro o detrás de nuestra pandereta y nos preocupamos poco de lo que es el hogar de todos que es la ciudad.
Por eso esta ley establece que cuando un cable está en desuso, la empresa que lo instaló y que lo ocupó se tiene que hacer cargo de sacando, y para eso va a tener un plazo y si no cumple dentro los plazos, la municipalidad va a poder sacarlo y, por supuesto, pasarle la cuenta a la empresa que instaló ese cable.
Así que estoy muy contento de poder compartir con ustedes hoy día la promulgación de esta ley, que nos va a permitir acelerar el proceso de tener ciudades para la gente, para la familia, para los niños, para los “Adultos Mejores”, para todos.
Y por eso hoy día es un día en que estamos empezando a hacernos cargo de que la calidad de vida no es solamente producción e inversión, la calidad de vida tiene mucho que ver con la calidad de las familias y con la calidad de nuestras ciudades.
Por eso, el Proyecto de Ley de Flexibilida

Calculemos la similitud del centroide del resumen con el centroide mean_vec.

In [78]:
centroide_resumen = 0
for i in resumen:
  vec_resumen = ObtenerEmbeddingOracion(modelo, i)
  centroide_resumen += (1-cosine(vec_resumen,mean_vec))

b = centroide_resumen/len(resumen)
print(b)

0.9264084696769714


Ahora hagamos un resumen para un texto que no haya sido usado para entrenar el modelo.

Primero cargamos nuevos discursos que no hayan sido utilizado en el modelo entrenado.

In [83]:
PATH_r = "/content/DiscursosResumibles/"
nlp          = es_core_news_sm.load()
corpus_r,docID_r = CrearCorpus(PATH_r)
oraciones_r    =  PreProcesarOraciones(corpus_r)
CorpusConClave_r  = CrearDiccionario(corpus_r,docID_r)

def Obtener_Puntajes(docID):
  vector_distance = []
  token_doc_r = nltk.sent_tokenize(CorpusConClave_r[docID])
  for j in token_doc_r:
    vector_distance.append(1 - cosine(mean_vec, ObtenerEmbeddingOracion(modelo, j)))
  return vector_distance

O = nltk.sent_tokenize(CorpusConClave_r['71541.txt'])
P = Obtener_Puntajes('71541.txt')
N = 20
U = 0.9
resumen = Generar_Resumen(O, P, N, U)

for i in resumen:
  print(i)

centroide_resumen = 0
for i in resumen:
  vec_resumen = ObtenerEmbeddingOracion(modelo, i)
  centroide_resumen += (1-cosine(vec_resumen,mean_vec))
b = centroide_resumen/len(resumen)
print(b)

Hoy, hemos querido que mi primera actividad como Presidente en ejercicio, sea visitar y compartir con los niños, las niñas y también con los adolescentes de este hogar del Sename, de la Fundación Padre Semería, en La Pintana.
Queridos compatriotas:

 

Sabemos que ya no podemos reparar la dolorosa pérdida de cientos de vidas y tampoco el sufrimiento causado, y la inocencia arrebatada a tantas niñas y niños en nuestro país.
Pero sí sabemos que podemos, debemos y vamos a dar a nuestros niños y adolescentes la prioridad y atención que ellos necesitan y merecen, para ponerlos primeros en la fila de las prioridades y en el centro de nuestro Gobierno.
En sus cortas vidas, muchos niños, niñas y adolescentes de los hogares del Sename o de organismos colaboradores, han sufrido demasiado, porque se les ha negado lo más importante que requiere todo ser humano, y especialmente en las primeras etapas de nuestras vidas, como es el amor, el acogimiento, la estimulación y la protección.
Segundo, reali

In [84]:
print(CorpusConClave_r['71541.txt'])

Muy buenas tardes:

 

La verdad es que para mí éste es un momento muy especial. Venimos, literalmente, llegando de la Transmisión del Mando Presidencial en Valparaíso y de un almuerzo con el Rey Emérito de España, los Presidentes y los Jefes de Estado -en Cerro Castillo- de los países amigos que nos han visitado.

 

Recuerdo muy bien que hace 8 años, mi primera actividad como Presidente fue visitar y compartir con las familias de las regiones que habían sido golpeadas por el devastador terremoto y maremoto del 27 de febrero, y también por las violentas réplicas que ocurrieron ese mismo día 11 de marzo cuando ocurría el cambio de mando.

 

Hoy, hemos querido que mi primera actividad como Presidente en ejercicio, sea visitar y compartir con los niños, las niñas y también con los adolescentes de este hogar del Sename, de la Fundación Padre Semería, en La Pintana.

 

Muchos de los niños que viven o asisten a los centros del Sename han sufrido la desintegración de sus familias, a veces 

## Testeando parametros.

A continuación, se testeo para los parametros se size y window en el modelo Word2vec. 

Se comparó la similitud entre el texto y el centroide, y el resumen y el centroide. Aquí basicamente se hizo lo que se hizo arriba.

In [64]:
def EntrenarModelo(oraciones,NombreModelo,n_size, n_window):
    model = Word2Vec(oraciones, size=n_size, window=n_window, min_count=1)
    model.save(NombreModelo)

def testing(n_size, n_window):

  EntrenarModelo(oraciones,'mi_word2vec', n_size, n_window)
  print(f"Modelo con {n_size} n_size y {n_window} n_window")
  modelo, vocabulario = CargarModelo('mi_word2vec')

  vectores = []
  for i in docID:
    doc = CorpusConClave[i]
    vec = ObtenerEmbeddingOracion(modelo, doc)
    vectores.append(vec)

  suma_vec = vectores[0] - vectores[0]
  for j in vectores:
    suma_vec += j
  mean_vec = suma_vec/len(vectores)

  O = nltk.sent_tokenize(CorpusConClave['100020.txt'])
  P = Obtener_Puntajes('100020.txt')
  N = 15
  U = 0.8
  resumen = Generar_Resumen(O, P, N, U)

  doc = CorpusConClave['100020.txt']
  vec = ObtenerEmbeddingOracion(modelo, doc)
  similitud = 1-cosine(vec,mean_vec)
  #print("Similitud centroide texto y centroide total")
  #print(similitud)

  centroide_resumen = 0
  for i in resumen:
    vec_resumen = ObtenerEmbeddingOracion(modelo, i)
    centroide_resumen += (1-cosine(vec_resumen,mean_vec))

  #print("Similitud centroide resumen y centroide total")
  #print(centroide_resumen/len(resumen))

  b = centroide_resumen/len(resumen)
  return b, n_size, n_window

Se usan varios valores para los parametros size y window. En este caso se determina como mejor al modelo que más similitud tenga el resumen generado con el centroide de los discursos.

In [65]:
n_size = [100, 200, 300, 500, 700]
window_size = [2, 3, 4, 5, 10]

best_score = 0
best_n_size = 0
best_window_size = 0

for i in n_size:
  for j in window_size:
    score, size_1, size_2 = testing(i, j)
    if score > best_score:
      best_score = score
      best_n_size = size_1
      best_window_size = size_2

print("------------------------------------")
print(f"El mejor modelo es con {best_n_size} n_size y {best_window_size} window_size. Con un score de {best_score}")

Modelo con 100 n_size y 2 n_window
Modelo con 100 n_size y 3 n_window
Modelo con 100 n_size y 4 n_window
Modelo con 100 n_size y 5 n_window
Modelo con 100 n_size y 10 n_window
Modelo con 200 n_size y 2 n_window
Modelo con 200 n_size y 3 n_window
Modelo con 200 n_size y 4 n_window
Modelo con 200 n_size y 5 n_window
Modelo con 200 n_size y 10 n_window
Modelo con 300 n_size y 2 n_window
Modelo con 300 n_size y 3 n_window
Modelo con 300 n_size y 4 n_window
Modelo con 300 n_size y 5 n_window
Modelo con 300 n_size y 10 n_window
Modelo con 500 n_size y 2 n_window
Modelo con 500 n_size y 3 n_window
Modelo con 500 n_size y 4 n_window
Modelo con 500 n_size y 5 n_window
Modelo con 500 n_size y 10 n_window
Modelo con 700 n_size y 2 n_window
Modelo con 700 n_size y 3 n_window
Modelo con 700 n_size y 4 n_window
Modelo con 700 n_size y 5 n_window
Modelo con 700 n_size y 10 n_window
------------------------------------
El mejor modelo es con 700 n_size y 2 window_size. Con un score de 0.93127183119455